# SloughGPT Training on Google Colab

Train your own SloughGPT model with GPU acceleration!

## Quick Start

In [ ]:
# Clone the repository
!git clone https://github.com/iamtowbee/sloughGPT.git
%cd sloughGPT

In [ ]:
# Install dependencies
!pip install torch numpy

## Download/Create Training Data

In [ ]:
# Create datasets directory and download training data
from pathlib import Path
import urllib.request

datasets_dir = Path("datasets")
datasets_dir.mkdir(exist_ok=True)

# Download Karpathy's tiny Shakespeare dataset
corpus_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
corpus_path = datasets_dir / "shakespeare.txt"

if not corpus_path.exists():
    print("Downloading Shakespeare dataset...")
    urllib.request.urlretrieve(corpus_url, corpus_path)
    print(f"Downloaded to: {corpus_path}")
else:
    print(f"Dataset already exists: {corpus_path}")

# Read and display dataset info
with open(corpus_path) as f:
    text = f.read()
print(f"Dataset size: {len(text):,} characters")
print(f"First 100 chars: {text[:100]}")

## Training Configuration

In [ ]:
import torch
import sys
sys.path.insert(0, '.')

from domains.training import TrainingConfig, Trainer

# Check GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Training on: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## Train Model

In [ ]:
# Configuration - adjust these values
CONFIG = {
    "data_path": "datasets/shakespeare.txt",
    "epochs": 10,
    "batch_size": 64,
    "n_embed": 256,
    "n_layer": 6,
    "n_head": 8,
    "vocab_size": 5000,
    "learning_rate": 1e-3,
}

# Create config
config = TrainingConfig(
    data_path=CONFIG["data_path"],
    epochs=CONFIG["epochs"],
    batch_size=CONFIG["batch_size"],
    n_embed=CONFIG["n_embed"],
    n_layer=CONFIG["n_layer"],
    vocab_size=CONFIG["vocab_size"],
    learning_rate=CONFIG["learning_rate"],
    max_batches=500,
)

print(f"Config: {CONFIG}")

In [ ]:
# Create trainer and train
print("Creating trainer...")
trainer = Trainer(config)
trainer.setup()

num_params = sum(p.numel() for p in trainer.model.model.parameters())
print(f"Model parameters: {num_params:,}")

# Train
print("\nStarting training...")
results = trainer.train()
print(f"\nTraining complete! Results: {results}")

## Save Model

In [ ]:
from pathlib import Path

# Save trained model
output_dir = Path("models/sloughgpt")
output_dir.mkdir(parents=True, exist_ok=True)

torch.save({
    'model_state_dict': trainer.model.model.state_dict(),
    'config': CONFIG,
}, output_dir / "sloughgpt_trained.pt")

print(f"Model saved to: {output_dir / 'sloughgpt_trained.pt'}")

# Download to local machine
from google.colab import files
files.download(output_dir / "sloughgpt_trained.pt")

## Generate Text (Test Model)

In [ ]:
# Training complete!
print("Training complete! Model saved.")